In [1]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import pandas as pd
import warnings
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor
import string
from multiprocessing import Pool
from concurrent.futures import ThreadPoolExecutor
from multiprocessing.pool import ThreadPool
warnings.filterwarnings("ignore")
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [2]:
def get_soup(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "lxml")
    return soup


def extract_links(url):
    soup = get_soup(url)
    aitems = []
    for h3 in soup.find_all("h3"):
        try:
            a = h3.a["href"]
        except:
            KeyError
        aitems.append(a)
    return aitems


all_links1 = [
    f"https://xwatch.vn/dong-ho-nam-pc85-page{idx_page}.html"
    for idx_page in range(1, 206)
]
all_links2 = [
    f"https://xwatch.vn/dong-ho-nu-pc86-page{idx_page}.html"
    for idx_page in range(1, 107)
]

all_links = []

with concurrent.futures.ThreadPoolExecutor(max_workers=25) as executor:
    futures = [executor.submit(extract_links, url) for url in all_links1 + all_links2]
    results = [future.result() for future in concurrent.futures.as_completed(futures)]

all_links = [link for sublist in results for link in sublist]

In [ ]:
all_links = list(set(all_links))

In [ ]:
def get_soup(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
    }

    with requests.Session() as session:
        response = session.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
    return soup


soup = get_soup(all_links[0])
details = {}
nameAndtype = soup.find("div", class_="product_name")
ds = nameAndtype.getText().lower().replace("chính hãng","").strip().split(" ")
type = ds[-1].strip().upper()
price = (
    soup.find_all("h3", class_="price_current")[0]
    .text.replace("₫", "")
    .strip()
)

content = soup.find("div", class_="table-condensed compare_table")
rows = content.find_all("tr")
for row in rows:
    text = row.getText().replace("\t", "").replace("\n", "").replace("\r", "")
    if len(text.split(":")) == 2:
        key, value = text.split(":")
        details[key.lower().strip()] = value.strip()
a = {"Giá tiền": price, "Mã sản phẩm": type, **details}
print(a)

In [7]:
def get_soup(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
    }

    with requests.Session() as session:
        response = session.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
    return soup
def get_product_info(url):
    soup = get_soup(url)
    details = {}
    nameAndtype = soup.find("div", class_="product_name")
    ds = nameAndtype.getText().lower().replace("chính hãng","").strip().split(" ")
    type = ds[-1].strip().upper()
    price = (
        soup.find_all("h3", class_="price_current")[0]
        .text.replace("₫", "")
        .strip()
    )

    content = soup.find("div", class_="table-condensed compare_table")
    rows = content.find_all("tr")
    for row in rows:
        text = row.getText().replace("\t", "").replace("\n", "").replace("\r", "")
        if len(text.split(":")) == 2:
            key, value = text.split(":")
            details[key.strip()] = value.strip()
    a = {"Giá tiền": price, "Mã sản phẩm": type, **details}
    print(a)
    return {"Giá tiền": price, "Mã sản phẩm": type, **details}

In [8]:
def crawl_data_from_web(all_links):
    data_frame_watchs = pd.DataFrame()
    with ThreadPoolExecutor(max_workers=25) as executor:
        results = executor.map(get_product_info, all_links)
        for result in results:
            data_frame_watchs = data_frame_watchs.append(result, ignore_index=True)
    return data_frame_watchs

data_frame_watchs = crawl_data_from_web(all_links)

data_frame_watchs.to_csv("../main/raw_data/x_watch.csv")